In [ ]:
# Global data variables
SANDBOX_NAME = ''# Sandbox Name
DATA_PATH = "/data/sandboxes/" + SANDBOX_NAME + "/data/data/" 



# Valores Ausentes

Los valores ausentes en _pyspark_ están identificados como _null_. El método `isNull` permite idenficar los registros nulos y `isNotNull` los no nulos.

In [ ]:
from pyspark.sql import functions as F

In [ ]:
vancouver_df = spark.read.csv(DATA_PATH + 'crime_in_vancouver.csv', sep=',', header=True, inferSchema=True)

In [ ]:
vancouver_df.filter(F.col('NEIGHBOURHOOD').isNull()).show(4)

In [ ]:
vancouver_df.filter(F.col('NEIGHBOURHOOD').isNotNull()).show(4)

 

## Conteo de valores nulos

In [ ]:
vancouver_df.filter(F.col('NEIGHBOURHOOD').isNull()).count()

In [ ]:
vancouver_df.filter(F.col('TYPE').isNull()).count()



### Porcentaje de ausentes por columna

El primer método es menos eficiente que el segundo ya que requiere ejecutar una acción por cada columna. Como norma general en Spark hay que intentar realizar el número mínimo de acciones.

In [ ]:
n_rows_vancouver = vancouver_df.count()



__Método 1:__

In [ ]:
%%time

for col in vancouver_df.columns:
    
    n_missing = vancouver_df.filter(F.col(col).isNull()).count()
    perc_missing = 100 * n_missing / n_rows_vancouver
    
    print(col, round(perc_missing, 2))



__Método 2:__

Para una única columna

In [ ]:
vancouver_df.select(F.round(F.sum(F.col('NEIGHBOURHOOD').isNull().cast('int')) * 100 / n_rows_vancouver, 2)\
                      .alias('NEIGHBOURHOOD')).show()



Todas las columnas

In [ ]:
%%time 

missing_ops = [F.round(F.sum(F.col(c).isNull().cast('int')) * 100 / n_rows_vancouver, 2).alias(c) 
               for c in vancouver_df.columns]

vancouver_df.select(missing_ops).show()

 

## Eliminación registros nulos

El método `dropna` se utiliza para eliminar registros nulos. Con el parámetro `subset` se indican sobre qué columnas buscar nulos y el parámetro `how` selecciona con qué condición se elimina un registro. Por defecto, `how` está a 'any'.

In [ ]:
vancouver_df.dropna(how='all').count()

In [ ]:
vancouver_df.dropna(how='any').count()

In [ ]:
vancouver_no_missing_df = vancouver_df.dropna(subset=['HOUR', 'MINUTE'])

In [ ]:
vancouver_no_missing_df.select(missing_ops).show()



## Imputación de valores nulos

`fillna` imputa los valores nulos de las columnas a un valor fijo elegido.

In [ ]:
vancouver_df.show(3)



Imputa los valores nulos de las columnas `HOUR` y `MINUTE` por el valor 0, y los de la columna `NEIGHBOURHOOD` por 'Unknown'.

In [ ]:
vancouver_df.fillna(0, subset=['HOUR', 'MINUTE']).show(3)

In [ ]:
vancouver_df.fillna('Unknown', subset=['NEIGHBOURHOOD']).show(3)



## Ejercicio 1



Usando el siguiente dataframe

In [ ]:
vancouver_df = spark.read.csv(DATA_PATH + 'crime_in_vancouver.csv', sep=',', header=True, inferSchema=True)



- a. Determine que columna(s) tiene(n) el mayor número de nulos
- b. Complete las variables categóricas con nulos con el valor mayoritario
- c. Elimine los registros con mayor número de nulos
- d. Complete las variables cuantitativas con nulos con los valores medios correspondientes de esas columnas

In [ ]:
# Respuesta

n_rows_vancouver = vancouver_df.count()

missing_ops = [F.round(F.sum(F.col(c).isNull().cast('int')) * 100 / n_rows_vancouver, 2).alias(c+"_PORCENTAJE_NULOS") 
               for c in vancouver_df.columns]

vancouver_df.select(missing_ops).show()

vancouver_df.printSchema()

In [ ]:
# Respuesta

most_frequent_neighbourhood = vancouver_df.groupBy('NEIGHBOURHOOD').count().sort('count', ascending=False).first()['NEIGHBOURHOOD']                               
vancouver_df.fillna(most_frequent_neighbourhood, subset=['NEIGHBOURHOOD']).show(3)

In [ ]:
# Respuesta

vancouver_df.withColumn('num_nulls', sum(vancouver_df[col].isNull().cast('int') for col in vancouver_df.columns)).show()

# Find the highest number of missing values in the registries
max_nulls = vancouver_df.withColumn('num_nulls', sum(vancouver_df[col].isNull().cast('int') for col in vancouver_df.columns)).select('num_nulls').sort(F.desc('num_nulls')).first()[0]

# Total number of columns
num_col = len(vancouver_df.columns)

# Set the limit for null removals per row
limit = num_col - max_nulls + 1 

# Delete those registries with the most missing values
print("Number of rows after dropna: " + str(vancouver_df.dropna(thresh=limit).count()))
print("Number of initial rows: " + str(vancouver_df.count()))

In [ ]:
# Respuesta

mean_hour = vancouver_df.agg(F.mean('HOUR')).first()[0]
mean_minute = vancouver_df.agg(F.mean('MINUTE')).first()[0]

vancouver_df.fillna(mean_hour, subset=['HOUR']).show(3)
vancouver_df.fillna(mean_minute, subset=['MINUTE']).show(3)



## Ejercicio 2

Fuente de los datos: https://www.kaggle.com/abhinav89/telecom-customer

1) Obtener un diccionario de las variables con el valor del porcentaje de nulos que contengan. Ordenarlo, de alguna forma aunque la salida no sea un diccionario, de mayor a menor porcentaje de nulos.

2) Realiza el tratamiento que consideres para los datos nulos, en función del significado de negocio que consideres para cada caso y la cantidad de datos nulos que contenga la columna. Imputar al menos cinco columnas a modo de ejemplo, justificando los valores sustituidos a nivel de negocio.

Hint: consideraremos que la columna no aporta valor si contiene más del 40% de sus valores nulos


In [ ]:
df = spark.read.csv(DATA_PATH + 'telecom_customer_churn.csv', sep=',', header=True, inferSchema=True)

In [ ]:
df.count()



1) Obtener un diccionario de las variables con el valor del porcentaje de nulos que contengan. Ordenarlo, de alguna forma aunque la salida no sea un diccionario, de mayor a menor porcentaje de nulos.

In [ ]:
# Respuesta

import pyspark.sql.functions as F
missing_ops = [F.round(F.sum(F.col(c).isNull().cast('int')) * 100 / df.count(), 2).alias(c) 
               for c in df.columns]

In [ ]:
# Respuesta

null_values = df.select(missing_ops).first()

In [ ]:
# Respuesta

with_null_values={}
for i, value in enumerate(null_values):
    if value!=0:
        with_null_values[df.columns[i]]=value


In [ ]:
# Respuesta

sorted(with_null_values.items(), key=lambda x: x[1], reverse=True)



2) Realiza el tratamiento que consideres para los datos nulos, en función del significado de negocio que consideres para cada caso y la cantidad de datos nulos que contenga la columna. Imputar al menos cinco columnas a modo de ejemplo, justificando los valores sustituidos a nivel de negocio.

Hint: consideraremos que la columna no aporta valor si contiene más del 40% de sus valores nulos

In [ ]:
# Respuesta

# First we drop those variables that contain more then 40% nulls

for x in with_null_values.items():
    if x[1]>40:
        print("Se va a eliminar", x[0])
        df = df.drop(F.col(x[0]))
        with_null_values.pop(x[0])

In [ ]:
# Respuesta

# If values cant be imputed and they are less than 40%

fill_cols_vals = {
"rev_Mean": 0,
"mou_Mean": 0,
"totmrc_Mean": 0,
"da_Mean": 0,
"ovrmou_Mean": 0,
"ovrrev_Mean": 0,
"vceovr_Mean": 0,
"datovr_Mean": 0,
"roam_Mean": 0,
"change_mou": 0,
"change_rev": 0,
"eqpdays": 0,
"forgntvl": 0,
"avg6qty": 0,
"avg6rev": 0,
"avg6mou": 0,
"kid16_17": "U",
"kid11_15": "U",
"kid6_10": "U",
"kid3_5": "U",
"kid0_2": "U",
}

df = df.na.fill(fill_cols_vals)

df = df.na.drop("any", subset=["HHstatin", "dwllsize", "creditcd", "ownrent", "marital", 
                               "rv", "truck", "hnd_webcap", "models", 
                                "phones", "hnd_price", "refurb_new", "dualband", 
                               "area", "prizm_social_one", "dwlltype", "lor",
                              "income", "adults", "infobase"])